In [1]:
import gc  
import os  
import time  
import math
import warnings 
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from functools import partial
from itertools import combinations  
from warnings import simplefilter 
import joblib  
import lightgbm as lgb  
import numpy as np  
import pandas as pd  
from sklearn.metrics import mean_absolute_error, mean_squared_error 
from sklearn.model_selection import KFold, TimeSeriesSplit  
import polars as pl
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

is_offline = False 
LGB = True
NN = False
is_train = True  
is_infer = True 
max_lookback = np.nan 
split_day = 435  

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


# **Utilities**

In [2]:
def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return w

In [3]:
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [4]:
def reduce_mem_usage(df, verbose=0):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
               
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    if verbose:
        logger.info(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        logger.info(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        logger.info(f"Decreased by {decrease:.2f}%")
    return df


# Data Loading






In [5]:
df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
df = df[~np.isnan(df["reference_price"])]
# df = df.dropna(subset=["target"])
df.reset_index(drop=True, inplace=True)
df.isna().sum()

stock_id                         0
date_id                          0
seconds_in_bucket                0
imbalance_size                   0
imbalance_buy_sell_flag          0
reference_price                  0
matched_size                     0
far_price                  2894122
near_price                 2856960
bid_price                        0
bid_size                         0
ask_price                        0
ask_size                         0
wap                              0
target                           0
time_id                          0
row_id                           0
dtype: int64

In [6]:
# 取得分群結果
labels = [0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0,
 0, 3, 0, 0, 0, 0, 0, 0, 4, 3, 0, 1, 3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0, 3, 3,
 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0]

len(labels)

200

In [7]:
# 存取分群結果
label = {i:labels[i] for i in sorted(df["stock_id"].unique())}
df["cluster"] = [label[id] for id in df["stock_id"]]

In [8]:
deal_col = list(df.columns[[3, 5, 6, 7, 8, 9, 10, 11, 12, 13]])
df["imbalance_size"] += 1

def calculate_Volatility(dataframe, deal_columns, seconds_diff):
    deal_columns_new = ["new_"+col+str(seconds_diff) for col in deal_columns]
    new_dataframe = dataframe.copy()
    new_dataframe[deal_columns_new] = new_dataframe[deal_columns]
    seconds = sorted(dataframe["seconds_in_bucket"].unique(), reverse=True)
    for sec in seconds:
        if sec >= seconds_diff:
            p = new_dataframe[new_dataframe["seconds_in_bucket"]==sec-seconds_diff]
            n = new_dataframe[new_dataframe["seconds_in_bucket"]==sec]
            p.index = n.index
            new_dataframe.loc[n.index, deal_columns_new] = n[deal_columns_new]/p[deal_columns_new]
            
        else:
            sec_diff_index = new_dataframe[new_dataframe["seconds_in_bucket"]==sec].index
            new_dataframe.loc[sec_diff_index, deal_columns_new] = np.nan
            
    return new_dataframe

df = calculate_Volatility(dataframe=df, deal_columns=deal_col, seconds_diff=10)
df = calculate_Volatility(dataframe=df, deal_columns=deal_col, seconds_diff=20)
df_shape = df.shape
df_shape
df.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id', 'cluster',
       'new_imbalance_size10', 'new_reference_price10', 'new_matched_size10',
       'new_far_price10', 'new_near_price10', 'new_bid_price10',
       'new_bid_size10', 'new_ask_price10', 'new_ask_size10', 'new_wap10',
       'new_imbalance_size20', 'new_reference_price20', 'new_matched_size20',
       'new_far_price20', 'new_near_price20', 'new_bid_price20',
       'new_bid_size20', 'new_ask_price20', 'new_ask_size20', 'new_wap20'],
      dtype='object')

**Outlier removal**

 # Parallel Triplet Imbalance Calculation

In [9]:
from numba import njit, prange

@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

def calculate_triplet_imbalance_numba(price, df):
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]
    features_array = compute_triplet_imbalance(df_values, comb_indices)
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)
    return features


# Feature Generation Functions 

In [10]:
def imbalance_features(df):
    # Define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")

    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")

    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    
    df["stock_weights"] = df["stock_id"].map(weights)
    df["weighted_wap"] = df["stock_weights"] * df["wap"]
    df['wap_momentum'] = df.groupby('stock_id')['weighted_wap'].pct_change(periods=6)
   
#     df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
#     df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    
    df['spread_depth_ratio'] = (df['ask_price'] - df['bid_price']) / (df['bid_size'] + df['ask_size'])
#     df['mid_price_movement'] = df['mid_price'].diff(periods=5).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    
    df['micro_price'] = ((df['bid_price'] * df['ask_size']) + (df['ask_price'] * df['bid_size'])) / (df['bid_size'] + df['ask_size'])
    df['relative_spread'] = (df['ask_price'] - df['bid_price']) / df['wap']
    
    # Calculate various statistical aggregation features
#     for func in ["mean", "std", "skew", "kurt"]:
#         df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
#         df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        

#     for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
#         for window in [1,3,5,10]:
#             df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
#             df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
    
#     # Calculate diff features for specific columns
#     for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size', 'weighted_wap','price_spread']:
#         for window in [1,3,5,10]:
#             df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)
    
#     #V4 feature
#     for window in [3,5,10]:
#         df[f'price_change_diff_{window}'] = df[f'bid_price_diff_{window}'] - df[f'ask_price_diff_{window}']
#         df[f'size_change_diff_{window}'] = df[f'bid_size_diff_{window}'] - df[f'ask_size_diff_{window}']

#     #V5 - rolling diff
#     # Convert from pandas to Polars
#     pl_df = pl.from_pandas(df)

#     #Define the windows and columns for which you want to calculate the rolling statistics
#     windows = [3, 5, 10]
#     columns = ['ask_price', 'bid_price', 'ask_size', 'bid_size']

#     # prepare the operations for each column and window
#     group = ["stock_id"]
#     expressions = []

#     # Loop over each window and column to create the rolling mean and std expressions
#     for window in windows:
#         for col in columns:
#             rolling_mean_expr = (
#                 pl.col(f"{col}_diff_{window}")
#                 .rolling_mean(window)
#                 .over(group)
#                 .alias(f'rolling_diff_{col}_{window}')
#             )

#             rolling_std_expr = (
#                 pl.col(f"{col}_diff_{window}")
#                 .rolling_std(window)
#                 .over(group)
#                 .alias(f'rolling_std_diff_{col}_{window}')
#             )

#             expressions.append(rolling_mean_expr)
#             expressions.append(rolling_std_expr)

#     # Run the operations using Polars' lazy API
#     lazy_df = pl_df.lazy().with_columns(expressions)

#     # Execute the lazy expressions and overwrite the pl_df variable
#     pl_df = lazy_df.collect()

#     # Convert back to pandas if necessary
#     df = pl_df.to_pandas()
#     gc.collect()
    
#     df['mid_price*volume'] = df['mid_price_movement'] * df['volume']
    df['harmonic_imbalance'] = df.eval('2 / ((1 / bid_size) + (1 / ask_size))')
    
    for col in df.columns:
        df[col] = df[col].replace([np.inf, -np.inf], 0)

    return df

# def other_features(df):
#     df["dow"] = df["date_id"] % 5  # Day of the week
#     df["seconds"] = df["seconds_in_bucket"] % 60  
#     df["minute"] = df["seconds_in_bucket"] // 60  
#     df['time_to_market_close'] = 540 - df['seconds_in_bucket']
    
#     for key, value in global_stock_id_feats.items():
#         df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

#     return df

def generate_all_features(df):
    # Select relevant columns for feature generation
#     cols = [c for c in df.columns if c not in ["row_id", "time_id", "target"]]
#     df = df[cols]
    
#     # Generate imbalance features
    df = imbalance_features(df)
#     gc.collect() 
#     df = other_features(df)
#     gc.collect()  
    feature_name = [i for i in df.columns if i not in ["row_id", "target", "time_id", "date_id"]]
    
    return df[feature_name]


In [11]:
weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]
weights = {int(k):v for k,v in enumerate(weights)}

## Data Splitting

In [12]:
if is_offline:
    
    df_train = df[df["date_id"] <= split_day]
    df_valid = df[df["date_id"] > split_day]
    print("Offline mode")
    print(f"train : {df_train.shape}, valid : {df_valid.shape}")
    
else:
    df_train = df
    print("Online mode")


Online mode


In [13]:
if is_train:
    global_stock_id_feats = {
        "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
        "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
        "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
        "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
    }
    if is_offline:
        df_train_feats = generate_all_features(df_train)
        print("Build Train Feats Finished.")
        df_valid_feats = generate_all_features(df_valid)
        print("Build Valid Feats Finished.")
        df_valid_feats = reduce_mem_usage(df_valid_feats)
    else:
        df_train_feats = generate_all_features(df_train)
        print("Build Online Train Feats Finished.")

    df_train_feats = reduce_mem_usage(df_train_feats)


Build Online Train Feats Finished.


# **Model Training**

# **LGB**

In [14]:

if LGB:
    import numpy as np
    import lightgbm as lgb
    
    lgb_params = {
        "objective": "mae",
        "n_estimators": 6000,
        "num_leaves": 256,
        "subsample": 0.6,
        "colsample_bytree": 0.8,
#         "learning_rate": 0.00871,
        "learning_rate": 0.01,
        'max_depth': 11,
        "n_jobs": 4,
        "device": "gpu",
        "verbosity": -1,
        "importance_type": "gain",
#         "reg_alpha": 0.1,
        "reg_alpha": 0.2,
        "reg_lambda": 3.25
    }

    feature_columns = list(df_train_feats.columns)
    print(f"Features = {len(feature_columns)}")
    #print(f"Feature length = {len(feature_columns)}")

    num_folds = 10
    fold_size = 480 // num_folds
    gap = 5

    models = []
    models_cbt = []
    scores = []

    model_save_path = 'modelitos_para_despues' 
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)

    date_ids = df_train['date_id'].values

    for i in range(num_folds):
        start = i * fold_size
        end = start + fold_size
        if i < num_folds - 1:  # No need to purge after the last fold
            purged_start = end - 2
            purged_end = end + gap + 2
            train_indices = (date_ids >= start) & (date_ids < purged_start) | (date_ids > purged_end)
        else:
            train_indices = (date_ids >= start) & (date_ids < end)

        test_indices = (date_ids >= end) & (date_ids < end + fold_size)
        
        gc.collect()
        
        df_fold_train = df_train_feats[train_indices]
        df_fold_train_target = df_train['target'][train_indices]
        df_fold_valid = df_train_feats[test_indices]
        df_fold_valid_target = df_train['target'][test_indices]

        print(f"Fold {i+1} Model Training")

        # Train a LightGBM model for the current fold
        lgb_model = lgb.LGBMRegressor(**lgb_params)
        lgb_model.fit(
            df_fold_train[feature_columns],
            df_fold_train_target,
            eval_set=[(df_fold_valid[feature_columns], df_fold_valid_target)],
            callbacks=[
                lgb.callback.early_stopping(stopping_rounds=100),
                lgb.callback.log_evaluation(period=100),
            ],
        )
        
#         cbt_model = cbt.CatBoostRegressor(objective='MAE', iterations=5000,bagging_temperature=0.5,
#                                 colsample_bylevel = 0.7,learning_rate = 0.065,
#                                 od_wait = 25,max_depth = 7,l2_leaf_reg = 1.5,
#                                 min_data_in_leaf = 1000,random_strength=0.65,
#                                 verbose=0,use_best_model=True,task_type='CPU')
#         cbt_model.fit(
#             df_fold_train[feature_columns],
#             df_fold_train_target,
#             eval_set=[(df_fold_valid[feature_columns], df_fold_valid_target)]
#         )
        
#         models_cbt.append(cbt_model)

        models.append(lgb_model)
        # Save the model to a file
        model_filename = os.path.join(model_save_path, f'doblez_{i+1}.txt')
        lgb_model.booster_.save_model(model_filename)
        print(f"Model for fold {i+1} saved to {model_filename}")

        # Evaluate model performance on the validation set
        #------------LGB--------------#
        fold_predictions = lgb_model.predict(df_fold_valid[feature_columns])
        fold_score = math.sqrt(mean_squared_error(fold_predictions, df_fold_valid_target))
        scores.append(fold_score)
        print(f":LGB Fold {i+1} RMSE: {fold_score}")
        #------------CBT--------------#
#         fold_predictions = cbt_model.predict(df_fold_valid[feature_columns])
#         fold_score_cbt = mean_absolute_error(fold_predictions, df_fold_valid_target)
#         scores.append(fold_score_cbt)
#         print(f"CBT Fold {i+1} MAE: {fold_score_cbt}")

        # Free up memory by deleting fold specific variables
        del df_fold_train, df_fold_train_target, df_fold_valid, df_fold_valid_target
        gc.collect()

    # Calculate the average best iteration from all regular folds
    average_best_iteration = int(np.mean([model.best_iteration_ for model in models]))

    # Update the lgb_params with the average best iteration
    final_model_params = lgb_params.copy()

    # final_model_params['n_estimators'] = average_best_iteration
    # print(f"Training final model with average best iteration: {average_best_iteration}")

    # Train the final model on the entire dataset
    num_model = 1

    for i in range(num_model):
        final_model = lgb.LGBMRegressor(**final_model_params)
        final_model.fit(
            df_train_feats[feature_columns],
            df_train['target'],
            callbacks=[
                lgb.callback.log_evaluation(period=100),
            ],
        )
        # Append the final model to the list of models
        models.append(final_model)


Features = 73
Fold 1 Model Training
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 6.07404
[200]	valid_0's l1: 6.04023
[300]	valid_0's l1: 6.02323
[400]	valid_0's l1: 6.0096
[500]	valid_0's l1: 5.999
[600]	valid_0's l1: 5.9903
[700]	valid_0's l1: 5.98256
[800]	valid_0's l1: 5.97528
[900]	valid_0's l1: 5.969
[1000]	valid_0's l1: 5.96334
[1100]	valid_0's l1: 5.9584
[1200]	valid_0's l1: 5.95368
[1300]	valid_0's l1: 5.94916
[1400]	valid_0's l1: 5.94481
[1500]	valid_0's l1: 5.94017
[1600]	valid_0's l1: 5.93628
[1700]	valid_0's l1: 5.93176
[1800]	valid_0's l1: 5.92708
[1900]	valid_0's l1: 5.92338
[2000]	valid_0's l1: 5.91923
[2100]	valid_0's l1: 5.91584
[2200]	valid_0's l1: 5.91261
[2300]	valid_0's l1: 5.90959
[2400]	valid_0's l1: 5.90649
[2500]	valid_0's l1: 5.90344
[2600]	valid_0's l1: 5.90059
[2700]	valid_0's l1: 5.89815
[2800]	valid_0's l1: 5.89567
[2900]	valid_0's l1: 5.89317
[3000]	valid_0's l1: 5.89096
[3100]	valid_0's l1: 5.88891
[3200]	valid_0's l1

# **Submission**

In [15]:
def seconds_0_calculate_Volatility(dataframe, deal_columns, column_number):
    deal_columns_new = ["new_"+col+str(column_number) for col in deal_columns]
    new_dataframe = dataframe.copy()
    new_dataframe[deal_columns_new] = new_dataframe[deal_columns]
    sec_0_index = new_dataframe.index
    new_dataframe.loc[sec_0_index, deal_columns_new] = np.nan
    return new_dataframe

def seconds_not0_calculate_Volatility(dataframe_new, dataframe_last, deal_columns, column_number):
    deal_columns_new = ["new_"+col+str(column_number) for col in deal_columns]
    new_dataframe = dataframe_new.copy()
    p = dataframe_last
    n = dataframe_new
    p.index = n.index
    new_dataframe[deal_columns_new] = n[deal_columns]/p[deal_columns]
    return new_dataframe
    

def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices) / np.sum(std_error)
    out = prices - std_error * step
    return out

if is_infer:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()
    counter = 0
    y_min, y_max = -64, 64
    qps, predictions = [], []
    pred = np.array([])

    # Weights for each fold model
    if LGB:
        lgb_model_weights = weighted_average(models)
        #cbt_model_weights = weighted_average(models_cbt)
        
    start_time = time.time()
    for (test, revealed_targets, sample_prediction) in iter_test:
        now_time = time.time()
        check_sec = test["seconds_in_bucket"].unique()[0]
        cache = test
        cache["cluster"] = [label[id] for id in cache["stock_id"]]
        # time gap = 10
        seconds_diff = 10
        if check_sec>=seconds_diff:
            cache = seconds_not0_calculate_Volatility(cache, test_past, deal_col, seconds_diff)
            test_past = test
        else:
            cache  = seconds_0_calculate_Volatility(cache, deal_col, seconds_diff)
            test_past = test
        
        # time gap = 20
        seconds_diff = 20
        if check_sec>=seconds_diff:
            cache = seconds_not0_calculate_Volatility(cache, test_past2, deal_col, seconds_diff)
            test_past2 = test_past1
            test_past1 = test
        elif check_sec==0:
            cache  = seconds_0_calculate_Volatility(cache, deal_col, seconds_diff)
            test_past1 = test
        else:
            cache  = seconds_0_calculate_Volatility(cache, deal_col, seconds_diff)
            if check_sec-test_past1["seconds_in_bucket"].unique()[0]<seconds_diff:
                test_past2 = test_past1
            test_past1 = test
        
        if counter > 0:
            cache = cache.groupby(['stock_id']).tail(21).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
        feat = generate_all_features(cache)# [-len(test):]
        print(f"Feat Shape is: {feat.shape}")

        # Generate predictions for each model and calculate the weighted average
        if LGB:
            lgb_predictions = np.zeros(len(test))
            for model, weight in zip(models, lgb_model_weights):
                lgb_predictions += weight * model.predict(feat[feature_columns])

        predictions = lgb_predictions
        
        #Using mean predictions rather than zero sum
        final_predictions = predictions - np.mean(predictions)
        clipped_predictions = np.clip(final_predictions, y_min, y_max)
#         pred = np.concatenate((pred, clipped_predictions))
        sample_prediction['target'] = clipped_predictions
        env.predict(sample_prediction)
        counter += 1
        qps.append(time.time() - now_time)
        if counter % 10 == 0:
            print(counter, 'qps:', np.mean(qps))
            
    
    time_cost = 1.146 * np.mean(qps)
    all_time = time.time()-start_time
    print(f"The code will take approximately {np.round(time_cost, 4)} hours to reason about")
    print(f"The code will take approximately {np.round(all_time, 4)} seconds to run")

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
10 qps: 0.9769087076187134
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
20 qps: 0.9134135246276855
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
Feat Shape is: (200, 74)
30 qps: 0.8925838390986125
Feat Shape is: (200, 74)
Feat Shape is: (200